In [1]:
import secrets
import sqlalchemy
import pandas as pd

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
DB_NAME_MAINNET = 'chainsync_qa_mainnet'
DB_NAME_PREPROD = 'chainsync_qa_preprod'
DB_NAME_PREVIEW = 'chainsync_qa_preview'

In [3]:
engine_mainnet = sqlalchemy.create_engine('postgresql://'+secrets.DB_USER+':'+secrets.DB_PASSWORD+'@'+secrets.DB_HOST+'/'+DB_NAME_MAINNET)
engine_preprod = sqlalchemy.create_engine('postgresql://'+secrets.DB_USER+':'+secrets.DB_PASSWORD+'@'+secrets.DB_HOST+'/'+DB_NAME_PREPROD)
engine_preview = sqlalchemy.create_engine('postgresql://'+secrets.DB_USER+':'+secrets.DB_PASSWORD+'@'+secrets.DB_HOST+'/'+DB_NAME_PREVIEW)

# trend_by_slots

In [4]:
df_mainnet = pd.read_sql('SELECT * FROM analysis.trend_by_slots', engine_mainnet)
df_preprod = pd.read_sql('SELECT * FROM analysis.trend_by_slots', engine_preprod)
df_preview = pd.read_sql('SELECT * FROM analysis.trend_by_slots', engine_preview)

In [5]:
def adjusted_slotno(val, offset):
    return int(val + offset)

df_mainnet['slotno_adjusted'] = df_mainnet.apply(lambda row: adjusted_slotno(row['slotno'], 1685297070 - 93730779),axis=1)
df_preprod['slotno_adjusted'] = df_preprod.apply(lambda row: adjusted_slotno(row['slotno'], 1685297096 - 29613896),axis=1)
df_preview['slotno_adjusted'] = df_preview.apply(lambda row: adjusted_slotno(row['slotno'], 1685297218 - 18641218),axis=1)

df_mainnet = df_mainnet.astype({'slotno_adjusted':'int'})
df_preprod = df_preprod.astype({'slotno_adjusted':'int'})
df_preview = df_preview.astype({'slotno_adjusted':'int'})

In [6]:
df_mainnet['date'] = pd.to_datetime(df_mainnet['slotno_adjusted'], unit='s').dt.date
df_preprod['date'] = pd.to_datetime(df_preprod['slotno_adjusted'], unit='s').dt.date
df_preview['date'] = pd.to_datetime(df_preview['slotno_adjusted'], unit='s').dt.date

In [7]:
df_mainnet = df_mainnet.groupby('date').tail(1).sort_values('slotno').reset_index(drop=True)
df_preprod = df_preprod.groupby('date').tail(1).sort_values('slotno').reset_index(drop=True)
df_preview = df_preview.groupby('date').tail(1).sort_values('slotno').reset_index(drop=True)

In [8]:
df_mainnet = df_mainnet.fillna(method='ffill')
df_preprod = df_preprod.fillna(method='ffill')
df_preview = df_preview.fillna(method='ffill')

/tmp/ipykernel_4207/3668841857.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_mainnet = df_mainnet.fillna(method='ffill')
/tmp/ipykernel_4207/3668841857.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_preprod = df_preprod.fillna(method='ffill')
/tmp/ipykernel_4207/3668841857.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_preview = df_preview.fillna(method='ffill')


In [9]:
df_mainnet = df_mainnet.groupby(['date'])[['creations','closures','active','transactions','ada_transacted','payment_addresses','stake_addresses']].max()
df_preprod = df_preprod.groupby(['date'])[['creations','closures','active','transactions','ada_transacted','payment_addresses','stake_addresses']].max()
df_preview = df_preview.groupby(['date'])[['creations','closures','active','transactions','ada_transacted','payment_addresses','stake_addresses']].max()

In [10]:
# each network are sorted and group by date
# now we merge all networks, and fill in the blanks

In [11]:
df_merged = pd.concat([df_mainnet, df_preprod, df_preview], axis=1)
df_merged = df_merged.sort_index()
# df_merged

In [12]:
df_merged = df_merged.fillna(method='ffill')
df_merged = df_merged.fillna(0)
# df_merged

/tmp/ipykernel_4207/3608558967.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged = df_merged.fillna(method='ffill')


In [13]:
df_merged.columns = [
    'mainnet_creations',
    'mainnet_closures',
    'mainnet_active',
    'mainnet_transactions',
    'mainnet_ada_transacted',
    'mainnet_payment_addresses',
    'mainnet_stake_addresses',
    'preprod_creations',
    'preprod_closures',
    'preprod_active',
    'preprod_transactions',
    'preprod_ada_transacted',
    'preprod_payment_addresses',
    'preprod_stake_addresses',
    'preview_creations',
    'preview_closures',
    'preview_active',
    'preview_transactions',
    'preview_ada_transacted',
    'preview_payment_addresses',
    'preview_stake_addresses',
]

In [14]:
df_mainnet2 = df_merged[[
    'mainnet_creations',
    'mainnet_closures',
    'mainnet_active',
    'mainnet_transactions',
    'mainnet_ada_transacted',
    'mainnet_payment_addresses',
    'mainnet_stake_addresses',
]]

df_preprod2 = df_merged[[
    'preprod_creations',
    'preprod_closures',
    'preprod_active',
    'preprod_transactions',
    'preprod_ada_transacted',
    'preprod_payment_addresses',
    'preprod_stake_addresses',
]]

df_preview2 = df_merged[[
    'preview_creations',
    'preview_closures',
    'preview_active',
    'preview_transactions',
    'preview_ada_transacted',
    'preview_payment_addresses',
    'preview_stake_addresses',
]]

df_mainnet2.columns = [
    'creations',
    'closures',
    'active',
    'transactions',
    'ada_transacted',
    'payment_addresses',
    'stake_addresses',
]
df_preprod2.columns = [
    'creations',
    'closures',
    'active',
    'transactions',
    'ada_transacted',
    'payment_addresses',
    'stake_addresses',
]
df_preview2.columns = [
    'creations',
    'closures',
    'active',
    'transactions',
    'ada_transacted',
    'payment_addresses',
    'stake_addresses',
]

df_mainnet2['network'] = 'mainnet'
df_preprod2['network'] = 'preprod'
df_preview2['network'] = 'preview'

/tmp/ipykernel_4207/1146035682.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainnet2['network'] = 'mainnet'
/tmp/ipykernel_4207/1146035682.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprod2['network'] = 'preprod'
/tmp/ipykernel_4207/1146035682.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [15]:
df_merged2 = pd.concat([df_mainnet2, df_preprod2, df_preview2])
# df_merged2

In [16]:
df_merged2 = df_merged2.astype({'creations':'int', 'closures':'int', 'active':'int', 'transactions':'int', 'ada_transacted':'int', 'payment_addresses':'int', 'stake_addresses':'int'})
# df_merged2

In [17]:
df_merged2.to_sql(name='merged_trend_by_slots', con=engine_mainnet, if_exists='replace', schema='analysis', index=True)

11

# contracts

In [18]:
df_mainnet = pd.read_sql('SELECT * FROM analysis.contracts', engine_mainnet)
df_preprod = pd.read_sql('SELECT * FROM analysis.contracts', engine_preprod)
df_preview = pd.read_sql('SELECT * FROM analysis.contracts', engine_preview)

In [19]:
def count_by_tag(df, network):
    tmp_df = df.groupby(['tag'])['id'].count()
    tmp_df = tmp_df.to_frame()
    tmp_df = tmp_df.rename(columns={"id": "count"})
    tmp_df['network'] = network
    tmp_df = tmp_df.reset_index()
    return tmp_df

df_mainnet = count_by_tag(df_mainnet, 'mainnet')
df_preprod = count_by_tag(df_preprod, 'preprod')
df_preview = count_by_tag(df_preview, 'preview')

In [20]:
df_merged = pd.concat([df_mainnet, df_preprod, df_preview])
df_merged


,tag,count,network
0,run-lite,17,mainnet
0,"""marlowe.examples.vesting.v0.0.5"";",1,preprod
1,5july2023,2811,preprod
2,CryptoPall2023,3,preprod
3,DAPP_TAG,2,preprod
...,...,...,...
16,run-lite-addr_test1qz7x6uvv4ay95udmxz4wv98m6g8...,2,preview
17,run-lite-addr_test1qzn6u4ffhafpfvsw876wxllvvae...,5,preview
18,swap.L1.by.marlowe.team,2,preview
19,swap.L1.dapp.by.marlowe.team,3,preview


In [21]:
df_merged.to_sql(name='merged_count_contracts_by_tags', con=engine_mainnet, if_exists='replace', schema='analysis', index=False)

92